# Setup

In [2]:
import numpy as np
from numpy import shape

from datetime import date

import random
import math
import dill
import glob
import gc

import matplotlib.pyplot as plt

from random import seed

from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow import device

# Tensorflow soll auf CPU und nicht auf der GPU laufen
device("cpu:0")
# für GPU:
# tf.device("gpu:0")

from tensorflow.keras.utils import to_categorical
from keras.layers import Dense
from keras.models import Sequential # Documentation: https://keras.io/models/sequential/

In [4]:
%run '/home/jovyan/rna/_functions/functions.py'

### Load RNASeq data

In [6]:
path_file = "/home/jovyan/rna/rna/data/agg_gene_data/agg_gene_data_short_ALL.csv"

X_genes = pd.read_csv(
        path_file, 
        header = 0,
        index_col = 0
    )

X_genes = X_genes.drop("DIAGNOSIS", axis=1)

# Subjects metadata
X_metadata_path = "/home/jovyan/rna/rna/data/agg_gene_data/agg_gene_metadata_ALL.csv"

X_metadata = pd.read_csv(
        X_metadata_path, 
        header = 0
)

X_genes = X_genes.merge(X_metadata[["sample", "DIAGNOSIS"]], "left", left_on = "id", right_on = "sample")

# Extract dependend variable
Y = X_genes["DIAGNOSIS"]
Y.value_counts()

PD         367
Control    171
Name: DIAGNOSIS, dtype: int64

In [7]:
# Labels faktorisieren
Y = pd.factorize(Y)[0]

X_genes = X_genes.drop(["sample","DIAGNOSIS", "id"], axis = 1)

print("0 = Control\n1 = PD")
print(np.unique(Y, return_counts=True))

0 = Control
1 = PD
(array([0, 1]), array([171, 367]))


### Load Persistence Landscapces

In [8]:
pl_folderpath = "/home/jovyan/rna/rna/data/persistence_landscapes_averages/"

# Parameter for persistence landscapes
pl_resolution = 1000
pl_num_landscapes = 10

In [9]:
# print loaded files
print(glob.glob(pl_folderpath + "avgPL_bucket01_H0_scaledWithin*.pkl")[-1])
print(glob.glob(pl_folderpath + "avgPL_bucket01_H1_scaledWithin*.pkl")[-1])
print(glob.glob(pl_folderpath + "avgPL_bucket01_H0_scaledBetween*.pkl")[-1])
print(glob.glob(pl_folderpath + "avgPL_bucket01_H1_scaledBetween*.pkl")[-1])
print(glob.glob(pl_folderpath + "avgPL_bucket01_H0_unscaled*.pkl")[-1])
print(glob.glob(pl_folderpath + "avgPL_bucket01_H1_unscaled*.pkl")[-1])

# Bucket01 scaledBetween H0 & H1
avgPL_bucket01_H0_scaledBetween = load_file(file = glob.glob(pl_folderpath + "avgPL_bucket01_H0_scaledBetween*.pkl")[-1])
avgPL_bucket01_H1_scaledBetween = load_file(file = glob.glob(pl_folderpath + "avgPL_bucket01_H1_scaledBetween*.pkl")[-1])
bucket01_H0_scaledBetween_rs = tf.reshape(avgPL_bucket01_H0_scaledBetween, [538, 10, 1000, 1])
bucket01_H1_scaledBetween_rs = tf.reshape(avgPL_bucket01_H1_scaledBetween, [538, 10, 1000, 1])

X_b01_scaledBetween_cnn = tf.concat(axis=3, values = [bucket01_H0_scaledBetween_rs, bucket01_H1_scaledBetween_rs]).numpy()

# Bucket01 scaledWithin H0 & H1
avgPL_bucket01_H0_scaledWithin = load_file(file = glob.glob(pl_folderpath + "avgPL_bucket01_H0_scaledWithin*.pkl")[-1])
avgPL_bucket01_H1_scaledWithin = load_file(file = glob.glob(pl_folderpath + "avgPL_bucket01_H1_scaledWithin*.pkl")[-1])
bucket01_H0_scaledWithin_rs = tf.reshape(avgPL_bucket01_H0_scaledWithin, [538, 10, 1000, 1])
bucket01_H1_scaledWithin_rs = tf.reshape(avgPL_bucket01_H1_scaledWithin, [538, 10, 1000, 1])

X_b01_scaledWithin_cnn = tf.concat(axis=3, values = [bucket01_H0_scaledWithin_rs, bucket01_H1_scaledWithin_rs]).numpy()

# Bucket01 unscaled H0 & H1
avgPL_bucket01_H0_unscaled = load_file(file = glob.glob(pl_folderpath + "avgPL_bucket01_H0_unscaled*.pkl")[-1])
avgPL_bucket01_H1_unscaled = load_file(file = glob.glob(pl_folderpath + "avgPL_bucket01_H1_unscaled*.pkl")[-1])
bucket01_H0_unscaled_rs = tf.reshape(avgPL_bucket01_H0_unscaled, [538, 10, 1000, 1])
bucket01_H1_unscaled_rs = tf.reshape(avgPL_bucket01_H1_unscaled, [538, 10, 1000, 1])

X_b01_unscaled_cnn = tf.concat(axis=3, values = [bucket01_H0_unscaled_rs, bucket01_H1_unscaled_rs]).numpy()

/home/jovyan/rna/rna/data/persistence_landscapes_averages/avgPL_bucket01_H0_scaledWithin_2022-03-05.pkl
/home/jovyan/rna/rna/data/persistence_landscapes_averages/avgPL_bucket01_H1_scaledWithin_2022-03-03.pkl
/home/jovyan/rna/rna/data/persistence_landscapes_averages/avgPL_bucket01_H0_scaledBetween_2022-03-04.pkl
/home/jovyan/rna/rna/data/persistence_landscapes_averages/avgPL_bucket01_H1_scaledBetween_2022-03-03.pkl
/home/jovyan/rna/rna/data/persistence_landscapes_averages/avgPL_bucket01_H0_unscaled_2022-03-03.pkl
/home/jovyan/rna/rna/data/persistence_landscapes_averages/avgPL_bucket01_H1_unscaled_2022-03-03.pkl


### Train-Test-Splits

In [10]:
seed(999)
X_cnn_b01_scaledBetween_train, X_cnn_b01_scaledBetween_test, y_cnn_b01_scaledBetween_train, y_cnn_b01_scaledBetween_test = train_test_split(X_b01_scaledBetween_cnn,
                                                                                                                                            Y,
                                                                                                                                            test_size = 0.2)

seed(999)
X_cnn_b01_scaledWithin_train, X_cnn_b01_scaledWithin_test, y_cnn_b01_scaledWithin_train, y_cnn_b01_scaledWithin_test = train_test_split(X_b01_scaledWithin_cnn,
                                                                                                                                        Y,
                                                                                                                                        test_size = 0.2)

seed(999)
X_cnn_b01_unscaled_train, X_cnn_b01_unscaled_test, y_cnn_b01_unscaled_train, y_cnn_b01_unscaled_test = train_test_split(X_b01_unscaled_cnn,
                                                                                                                        Y,
                                                                                                                        test_size = 0.2)

In [11]:
# Setze Seed, damit der Train-Test-Split bei den Gene-Daten der Gleiche ist wie bei den PL
seed(999)

# Split dataset into training set and test set
X_genes_train, X_genes_test, Y_train, Y_test = train_test_split(X_genes,
                                                                Y,
                                                                test_size=0.2)

print("Anteil PD-Ausprägungen in Gesamtdaten: ", sum(Y)/len(Y))
print("Anteil PD-Ausprägungen in Trainingsdaten: ", sum(Y_train)/len(Y_train))
print("Anteil PD-Ausprägungen in Testdaten: ", sum(Y_test)/len(Y_test))

Anteil PD-Ausprägungen in Gesamtdaten:  0.6821561338289963
Anteil PD-Ausprägungen in Trainingsdaten:  0.6790697674418604
Anteil PD-Ausprägungen in Testdaten:  0.6944444444444444


# Convolutional Neural Networks on Persistence Landscapes

In [12]:
# Folderpath to store the results
folderpath_results = "/home/jovyan/rna/rna/results/"

### CNN-Models

In [13]:
def CNN_model_paper():
    
    model_cnn = Sequential()

    model_cnn.add(Conv2D(64,
                        kernel_size=3, 
                        activation='elu', 
                        input_shape=(10, 1000, 2)
                       )
                )
    model_cnn.add(layers.MaxPool2D((2,2)))
    
    model_cnn.add(Conv2D(64, kernel_size=3, activation='elu'))
    model_cnn.add(layers.MaxPool2D((2, 2)))
    
    model_cnn.add(Flatten())
    model_cnn.add(Dense(32, activation='elu'))
    
    model_cnn.add(Flatten())
    model_cnn.add(Dense(1, activation='softmax'))
        
    return model_cnn

def CNN_model_paper2():
    
    model_cnn = Sequential()

    model_cnn.add(Conv2D(64,
                        kernel_size=3, 
                        activation='elu', 
                        input_shape=(10, 1000, 2),
                        padding='same'
                       )
                )
    model_cnn.add(layers.MaxPool2D((2)))
    
    model_cnn.add(Conv2D(64, kernel_size=3, activation='elu'))
    model_cnn.add(layers.MaxPool2D((2)))
    
    model_cnn.add(Flatten())
    model_cnn.add(Dense(32, activation='elu'))
    
    model_cnn.add(Flatten())
    model_cnn.add(Dense(1, activation='sigmoid'))
        
    return model_cnn

def CNN_model_paper_withDropout():
    
    model_cnn = Sequential()

    model_cnn.add(Conv2D(64,
                        kernel_size=3, 
                        activation='elu', 
                        input_shape=(10, 1000, 2),
                        padding='same'
                       )
                )
    model_cnn.add(layers.MaxPool2D((2)))
    
    model_cnn.add(Dropout(0.25))
    
    model_cnn.add(Conv2D(64, kernel_size=3, activation='elu'))
    model_cnn.add(layers.MaxPool2D((2)))
    
    model_cnn.add(Flatten())
    model_cnn.add(Dense(16, activation='elu'))
    
    model_cnn.add(Flatten())
    model_cnn.add(Dense(1, activation='softmax'))
        
    return model_cnn

def simple_CNN_model_1():
    
    model_cnn = Sequential()
    
    model_cnn.add(Conv2D(32,
                         kernel_size=3, 
                         activation='elu', 
                         input_shape=(10, 1000, 2),
                         padding='same'
                        )
                 )
    
    model_cnn.add(Flatten())
    
    model_cnn.add(Dense(16, activation='elu'))
    
    model_cnn.add(Flatten())
    
    model_cnn.add(Dense(1, activation='softmax'))
    
    return model_cnn

def simple_CNN_model_1_sigmoid():
    
    model_cnn = Sequential()
    
    model_cnn.add(Conv2D(32,
                         kernel_size=3, 
                         activation='elu', 
                         input_shape=(10, 1000, 2),
                         padding='same'
                        )
                 )
    
    model_cnn.add(Flatten())
    
    model_cnn.add(Dense(16, activation='elu'))
    
    model_cnn.add(Flatten())
    
    model_cnn.add(Dense(1, activation='sigmoid'))
    
    return model_cnn

def cnn_model_succeeded():

    model = Sequential()
    
    # add model layers
    model.add(Conv2D(64,
                     kernel_size=3,
                     activation='elu',
                     input_shape=(10,1000,2),
                     padding='same'
                    )
             )
    
    model.add(layers.MaxPool2D((2)))
    model.add(Flatten())
    model.add(Dense(32, activation='elu'))
    model.add(Dense(1, activation='sigmoid'))
    
    return model

In [14]:
cnn_models = [CNN_model_paper, CNN_model_paper2, CNN_model_paper_withDropout, simple_CNN_model_1, simple_CNN_model_1_sigmoid, cnn_model_succeeded]
cnn_learning_rates = [1e-3, 1e-4, 1e-5, 1e-6, 1e-7]
cnn_validation_splits = [0.33, None]
cnn_epochs = [1, 3, 5, 10, 20, 50, 100]

### Train CNN-Models on Persistence Landscapes (H1 & H0; scaledBetween)

In [15]:
grid_cnn_scaledBetween = create_modelgrid(models = cnn_models,
                                          learning_rates = cnn_learning_rates,
                                          validation_splits = cnn_validation_splits,
                                          epochs = cnn_epochs)

In [16]:
results_cnn_b01_scaledBetween = test_multiple_models(x_train = X_cnn_b01_scaledBetween_train,
                                                     y_train = y_cnn_b01_scaledBetween_train,
                                                     x_test = X_cnn_b01_scaledBetween_test,
                                                     y_test = y_cnn_b01_scaledBetween_test,
                                                     modelgrid = grid_cnn_scaledBetween)

Training model 1 of 420


2022-09-07 09:50:49.604121: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Training model 2 of 420
Training model 3 of 420
Training model 4 of 420
Training model 5 of 420
Training model 6 of 420
Training model 7 of 420
Training model 8 of 420
Training model 9 of 420
Training model 10 of 420
Training model 11 of 420
Training model 12 of 420
Training model 13 of 420
Training model 14 of 420
Training model 15 of 420
Training model 16 of 420
Training model 17 of 420
Training model 18 of 420
Training model 19 of 420
Training model 20 of 420
Training model 21 of 420
Training model 22 of 420
Training model 23 of 420
Training model 24 of 420
Training model 25 of 420
Training model 26 of 420
Training model 27 of 420
Training model 28 of 420
Training model 29 of 420
Training model 30 of 420
Training model 31 of 420
Training model 32 of 420
Training model 33 of 420
Training model 34 of 420
Training model 35 of 420
Training model 36 of 420
Training model 37 of 420
Training model 38 of 420
Training model 39 of 420
Training model 40 of 420
Training model 41 of 420
Training

Training model 322 of 420
Training model 323 of 420
Training model 324 of 420
Training model 325 of 420
Training model 326 of 420
Training model 327 of 420
Training model 328 of 420
Training model 329 of 420
Training model 330 of 420
Training model 331 of 420
Training model 332 of 420
Training model 333 of 420
Training model 334 of 420
Training model 335 of 420
Training model 336 of 420
Training model 337 of 420
Training model 338 of 420
Training model 339 of 420
Training model 340 of 420
Training model 341 of 420
Training model 342 of 420
Training model 343 of 420
Training model 344 of 420
Training model 345 of 420
Training model 346 of 420
Training model 347 of 420
Training model 348 of 420
Training model 349 of 420
Training model 350 of 420
Training model 351 of 420
Training model 352 of 420
Training model 353 of 420
Training model 354 of 420
Training model 355 of 420
Training model 356 of 420
Training model 357 of 420
Training model 358 of 420
Training model 359 of 420
Training mod

In [17]:
results_cnn_b01_scaledBetween = results_cnn_b01_scaledBetween.sort_values(by='accuracy', ascending=False).reset_index(drop=True)

# Save results to csv
results_cnn_b01_scaledBetween.to_csv(folderpath_results + "results_simple_CNN_b01_scaledBetween_H_0_1.csv",
                                              encoding='utf-8',
                                              index=False)

results_cnn_b01_scaledBetween.head(20)

,model,learning_rate,val_split,epochs,accuracy,TPR,TNR
0,simple_CNN_model_1_sigmoid,1.000000e-07,0.33,3,0.703704,0.972603,0.142857
1,cnn_model_succeeded,1.000000e-04,None,20,0.694444,0.890411,0.285714
2,CNN_model_paper2,1.000000e-07,None,1,0.685185,0.972603,0.085714
3,cnn_model_succeeded,1.000000e-03,0.33,1,0.685185,1.000000,0.028571
4,CNN_model_paper2,1.000000e-04,0.33,5,0.685185,0.931507,0.171429
5,cnn_model_succeeded,1.000000e-03,0.33,5,0.685185,1.000000,0.028571
6,CNN_model_paper2,1.000000e-07,None,5,0.685185,0.917808,0.200000
7,simple_CNN_model_1,1.000000e-03,0.33,20,0.675926,1.000000,0.000000
8,simple_CNN_model_1,1.000000e-04,0.33,20,0.675926,1.000000,0.000000
9,CNN_model_paper_withDropout,1.000000e-04,0.33,20,0.675926,1.000000,0.000000


### Train CNN-Models on Persistence Landscapes (H1 & H0; scaledWithin)

In [18]:
grid_cnn_scaledWithin = create_modelgrid(models = cnn_models,
                                         learning_rates = cnn_learning_rates,
                                         validation_splits = cnn_validation_splits,
                                         epochs = cnn_epochs)

In [19]:
results_cnn_b01_scaledWithin = test_multiple_models(x_train = X_cnn_b01_scaledWithin_train,
                                                    y_train = y_cnn_b01_scaledWithin_train,
                                                    x_test = X_cnn_b01_scaledWithin_test,
                                                    y_test = y_cnn_b01_scaledWithin_test,
                                                    modelgrid = grid_cnn_scaledWithin)

Training model 1 of 420
Training model 2 of 420
Training model 3 of 420
Training model 4 of 420
Training model 5 of 420
Training model 6 of 420
Training model 7 of 420
Training model 8 of 420
Training model 9 of 420
Training model 10 of 420
Training model 11 of 420
Training model 12 of 420
Training model 13 of 420
Training model 14 of 420
Training model 15 of 420
Training model 16 of 420
Training model 17 of 420
Training model 18 of 420
Training model 19 of 420
Training model 20 of 420
Training model 21 of 420
Training model 22 of 420
Training model 23 of 420
Training model 24 of 420
Training model 25 of 420
Training model 26 of 420
Training model 27 of 420
Training model 28 of 420
Training model 29 of 420
Training model 30 of 420
Training model 31 of 420
Training model 32 of 420
Training model 33 of 420
Training model 34 of 420
Training model 35 of 420
Training model 36 of 420
Training model 37 of 420
Training model 38 of 420
Training model 39 of 420
Training model 40 of 420
Training 

Training model 321 of 420
Training model 322 of 420
Training model 323 of 420
Training model 324 of 420
Training model 325 of 420
Training model 326 of 420
Training model 327 of 420
Training model 328 of 420
Training model 329 of 420
Training model 330 of 420
Training model 331 of 420
Training model 332 of 420
Training model 333 of 420
Training model 334 of 420
Training model 335 of 420
Training model 336 of 420
Training model 337 of 420
Training model 338 of 420
Training model 339 of 420
Training model 340 of 420
Training model 341 of 420
Training model 342 of 420
Training model 343 of 420
Training model 344 of 420
Training model 345 of 420
Training model 346 of 420
Training model 347 of 420
Training model 348 of 420
Training model 349 of 420
Training model 350 of 420
Training model 351 of 420
Training model 352 of 420
Training model 353 of 420
Training model 354 of 420
Training model 355 of 420
Training model 356 of 420
Training model 357 of 420
Training model 358 of 420
Training mod

In [20]:
results_cnn_b01_scaledWithin = results_cnn_b01_scaledWithin.sort_values(by='accuracy', ascending=False).reset_index(drop=True)

# Save results to csv
results_cnn_b01_scaledWithin.to_csv(folderpath_results + "results_simple_CNN_b01_scaledWithin_H_0_1.csv",
                                              encoding='utf-8',
                                              index=False)

results_cnn_b01_scaledWithin.head(20)

,model,learning_rate,val_split,epochs,accuracy,TPR,TNR
0,cnn_model_succeeded,1.000000e-07,None,10,0.703704,0.944444,0.222222
1,cnn_model_succeeded,1.000000e-03,0.33,20,0.675926,1.000000,0.027778
2,simple_CNN_model_1_sigmoid,1.000000e-03,0.33,100,0.675926,1.000000,0.027778
3,CNN_model_paper2,1.000000e-04,0.33,20,0.666667,1.000000,0.000000
4,simple_CNN_model_1,1.000000e-05,0.33,20,0.666667,1.000000,0.000000
5,CNN_model_paper_withDropout,1.000000e-05,0.33,20,0.666667,1.000000,0.000000
6,CNN_model_paper,1.000000e-05,0.33,20,0.666667,1.000000,0.000000
7,cnn_model_succeeded,1.000000e-04,0.33,20,0.666667,1.000000,0.000000
8,simple_CNN_model_1_sigmoid,1.000000e-04,0.33,20,0.666667,1.000000,0.000000
9,simple_CNN_model_1,1.000000e-04,0.33,20,0.666667,1.000000,0.000000


### Train CNN-Models on Persistence Landscapes (H1 & H0; unscaled)

In [21]:
grid_cnn_unscaled = create_modelgrid(models = cnn_models,
                                     learning_rates = cnn_learning_rates,
                                     validation_splits = cnn_validation_splits,
                                     epochs = cnn_epochs)

In [22]:
results_cnn_b01_unscaled = test_multiple_models(x_train = X_cnn_b01_unscaled_train,
                                                y_train = y_cnn_b01_unscaled_train,
                                                x_test = X_cnn_b01_unscaled_test,
                                                y_test = y_cnn_b01_unscaled_test,
                                                modelgrid = grid_cnn_unscaled)

Training model 1 of 420
Training model 2 of 420
Training model 3 of 420
Training model 4 of 420
Training model 5 of 420
Training model 6 of 420
Training model 7 of 420
Training model 8 of 420
Training model 9 of 420
Training model 10 of 420
Training model 11 of 420
Training model 12 of 420
Training model 13 of 420
Training model 14 of 420
Training model 15 of 420
Training model 16 of 420
Training model 17 of 420
Training model 18 of 420
Training model 19 of 420
Training model 20 of 420
Training model 21 of 420
Training model 22 of 420
Training model 23 of 420
Training model 24 of 420
Training model 25 of 420
Training model 26 of 420
Training model 27 of 420
Training model 28 of 420
Training model 29 of 420
Training model 30 of 420
Training model 31 of 420
Training model 32 of 420
Training model 33 of 420
Training model 34 of 420
Training model 35 of 420
Training model 36 of 420
Training model 37 of 420
Training model 38 of 420
Training model 39 of 420
Training model 40 of 420
Training 

Training model 321 of 420
Training model 322 of 420
Training model 323 of 420
Training model 324 of 420
Training model 325 of 420
Training model 326 of 420
Training model 327 of 420
Training model 328 of 420
Training model 329 of 420
Training model 330 of 420
Training model 331 of 420
Training model 332 of 420
Training model 333 of 420
Training model 334 of 420
Training model 335 of 420
Training model 336 of 420
Training model 337 of 420
Training model 338 of 420
Training model 339 of 420
Training model 340 of 420
Training model 341 of 420
Training model 342 of 420
Training model 343 of 420
Training model 344 of 420
Training model 345 of 420
Training model 346 of 420
Training model 347 of 420
Training model 348 of 420
Training model 349 of 420
Training model 350 of 420
Training model 351 of 420
Training model 352 of 420
Training model 353 of 420
Training model 354 of 420
Training model 355 of 420
Training model 356 of 420
Training model 357 of 420
Training model 358 of 420
Training mod

In [23]:
results_cnn_b01_unscaled = results_cnn_b01_unscaled.sort_values(by='accuracy', ascending=False).reset_index(drop=True)

# Save results to csv
results_cnn_b01_unscaled.to_csv(folderpath_results + "results_simple_CNN_b01_unscaled_H_0_1.csv",
                                              encoding='utf-8',
                                              index=False)

results_cnn_b01_unscaled.head(20)

,model,learning_rate,val_split,epochs,accuracy,TPR,TNR
0,CNN_model_paper2,1.000000e-07,None,100,0.722222,0.883117,0.322581
1,cnn_model_succeeded,1.000000e-06,0.33,50,0.722222,1.000000,0.032258
2,simple_CNN_model_1_sigmoid,1.000000e-06,None,5,0.722222,1.000000,0.032258
3,CNN_model_paper,1.000000e-03,0.33,1,0.712963,1.000000,0.000000
4,CNN_model_paper_withDropout,1.000000e-05,0.33,20,0.712963,1.000000,0.000000
5,CNN_model_paper_withDropout,1.000000e-07,0.33,20,0.712963,1.000000,0.000000
6,CNN_model_paper,1.000000e-07,0.33,20,0.712963,1.000000,0.000000
7,cnn_model_succeeded,1.000000e-06,0.33,20,0.712963,1.000000,0.000000
8,simple_CNN_model_1,1.000000e-06,0.33,20,0.712963,1.000000,0.000000
9,CNN_model_paper_withDropout,1.000000e-06,0.33,20,0.712963,1.000000,0.000000
